# Write tiles to COG Experimentation

## Define Raster Tile Template

In [ ]:
"""tests ml_export.tile_generator.base"""

import os
import pytest
## Note, for mac osx compatability import something from shapely.geometry before importing fiona or geopandas
## https://github.com/Toblerity/Shapely/issues/553  * Import shapely before rasterio or fioana
from shapely import geometry
import mercantile
from rio_tiler import main
import numpy as np
from ml_export import tile_generator, tile_aggregator
import rasterio
%load_ext autoreload
%autoreload 2

In [ ]:
## Define Raster Template
raster_tile_server_template = "https://14ffxwyw5l.execute-api.us-east-1.amazonaws.com/production/tiles/{z}/{x}/{y}.jpg?url=s3://spacenet-dataset/AOI_2_Vegas/srcData/rasterData/AOI_2_Vegas_MUL-PanSharpen_Cloud.tif&rgb=5,3,2&linearStretch=true&band1=5&band2=7&tilesize=256"
raster_tile_server_template = "https://tiles.openaerialmap.org/5ae36dd70b093000130afdd4/0/5ae36dd70b093000130afdd5/{z}/{x}/{y}.png"

In [ ]:
## Define Example Geometry

tile_obj = mercantile.tile(39.299515932798386, -6.080908028740757 , 12)

geom = geometry.box(-115.24, 36.1, -115.2, 36.2)
zoom_level = 12

large_tile_object_list = tile_generator.get_tile_list(geom, zoom=zoom_level)
print("{} z{} Tiles identified".format(len(large_tile_object_list), zoom_level))

In [ ]:
## Pick 1 Z Tile For For Testing and Get 2x Zoomed Tiles (z17 = 4x4 z19)
from tqdm import tqdm
file_name = "/Users/dlindenbaum/cosmiQGit/ml-export-tool/tests/fixtures/my-bucket/spacenet_test/las_vegas_tile_4.tiff"
raster_address = "s3://spacenet-dataset/AOI_2_Vegas/resultData/AOI_2_Vegas_MULPS_v13_cloud.tiff"
#raster_addrsss = 's3://spacenet-dataset/AOI_2_Vegas/srcData/rasterData/AOI_2_Vegas_MUL-PanSharpen_Cloud.tif'
small_tile_zoom=16
super_res_tile_zoom = 19
num_channels = 3
tile_obj = mercantile.tile(39.299515932798386, -6.080908028740757 , 14)

for large_tile_object in [tile_obj]:
    
    large_cog_profile = tile_aggregator.create_webmercator_cog_profile(large_tile_object, super_res_tile_zoom, num_channels=num_channels)
    print(large_cog_profile)
    with rasterio.open(file_name, 'w', **large_cog_profile) as dst_dataset:
        
        small_tile_object_list, small_tile_position_list = tile_generator.create_super_tile_list(large_tile_object, desired_zoom_level=small_tile_zoom)

        
        for small_tile_object in tqdm(small_tile_object_list):
            

            super_res_tile = tile_generator.create_super_tile_image(small_tile_object, raster_tile_server_template, 
                                                                    desired_zoom_level=super_res_tile_zoom, indexes=[1,2,3], 
                                                                    tile_size=256, cog=False)
            
            
            left, bottom, right, top = mercantile.xy_bounds(*small_tile_object)
            dst_window = rasterio.windows.from_bounds(left, bottom, right, top, transform=large_cog_profile['transform'])
            dst_transform = dst_dataset.window_transform(dst_window)
            dst_dataset.write(super_res_tile.astype(large_cog_profile['dtype']), window=dst_window)

            

In [ ]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")


class TileClassDataset(Dataset):
    
    def __init__(self, root_tile_obj, raster_location,
                 desired_zoom_level, super_res_zoom_level, 
                cog=True,
                tile_size=256,
                 indexes=None
                ):
    
        self.root_tile_obj = root_tile_obj
        self.desired_zoom_level = desired_zoom_level
        self.super_res_zoom_level = super_res_zoom_level
        self.raster_location = raster_location
        self.cog = cog
        self.tile_size = tile_size
        
        if indexes==None:
            self.indexes = [1,2,3]
        else:
            self.indexes = indexes
        
        
        small_tile_object_list, small_tile_position_list = tile_generator.create_super_tile_list(large_tile_object, desired_zoom_level=small_tile_zoom)
        self.small_tile_object_list = small_tile_object_list
        self.small_tile_position_list = small_tile_position_list
        
    def __len__(self):
        
        return len(self.small_tile_object_list)

    
    def __getitem__(self,idx):
    
        super_res_tile = tile_generator.create_super_tile_image(self.small_tile_object_list[idx], 
                                                                self.raster_location, 
                                                                desired_zoom_level=self.super_res_zoom_level,
                                                                indexes=self.indexes,
                                                                tile_size=self.tile_size, 
                                                                cog=self.cog)
        
        return super_res_tile, mercantile.xy_bounds(*self.small_tile_object_list[idx])
            

        
        
     

In [ ]:
%%time
## Pick 1 Z Tile For For Testing and Get 2x Zoomed Tiles (z17 = 4x4 z19)
from tqdm import tqdm
file_name = "/Users/dlindenbaum/cosmiQGit/ml-export-tool/tests/fixtures/my-bucket/spacenet_test/las_vegas_tile_5.tiff"
raster_address = "s3://spacenet-dataset/AOI_2_Vegas/resultData/AOI_2_Vegas_MULPS_v13_cloud.tiff"
#raster_addrsss = 's3://spacenet-dataset/AOI_2_Vegas/srcData/rasterData/AOI_2_Vegas_MUL-PanSharpen_Cloud.tif'
raster_tile_server_template = "https://tiles.openaerialmap.org/5ae36dd70b093000130afdd4/0/5ae36dd70b093000130afdd5/{z}/{x}/{y}.png"
small_tile_zoom=17
super_res_tile_zoom = 19
num_channels = 3
tile_obj = mercantile.tile(39.299515932798386, -6.080908028740757 , 14)

for large_tile_object in [tile_obj]:
    
    large_cog_profile = tile_aggregator.create_webmercator_cog_profile(large_tile_object, super_res_tile_zoom, num_channels=num_channels)
    print(large_cog_profile)
    with rasterio.open(file_name, 'w', **large_cog_profile) as dst_dataset:
        
        #small_tile_object_list, small_tile_position_list = tile_generator.create_super_tile_list(large_tile_object, desired_zoom_level=small_tile_zoom)

        TileIterator = TileClassDataset(root_tile_obj=large_tile_object, 
                         raster_location=raster_tile_server_template,
                         desired_zoom_level=small_tile_zoom, 
                         super_res_zoom_level=super_res_tile_zoom, 
                         cog=False,
                         tile_size=256,
                         indexes=None
                )
        dataloader = DataLoader(TileIterator, batch_size=4,
                        shuffle=False, num_workers=4)
                         
        for super_res_tile_batch, small_tile_obj_batch  in tqdm(dataloader):
            
            print(super_res_tile_batch.numpy().shape)
            
            for super_res_tile, small_tile_object_tensor in zip(super_res_tile_batch, small_tile_obj_batch):

                left, bottom, right, top = small_tile_object_tensor.numpy()
                dst_window = rasterio.windows.from_bounds(left, bottom, right, top, transform=large_cog_profile['transform'])
                dst_transform = dst_dataset.window_transform(dst_window)
                dst_dataset.write(super_res_tile.numpy().astype(large_cog_profile['dtype']), window=dst_window)

            #super_res_tile = tile_generator.create_super_tile_image(small_tile_object, raster_tile_server_template, 
            #                                                        desired_zoom_level=super_res_tile_zoom, indexes=[1,2,3], 
            #                                                        tile_size=256, cog=False)
            
            
            #left, bottom, right, top = mercantile.xy_bounds(*small_tile_object)
            #dst_window = rasterio.windows.from_bounds(left, bottom, right, top, transform=large_cog_profile['transform'])
            #dst_transform = dst_dataset.window_transform(dst_window)
            #dst_dataset.write(super_res_tile.astype(large_cog_profile['dtype']), window=dst_window)


In [ ]:
small_tile_obj_batch

In [ ]:
left, bottom, right, top = small_tile_object.numpy()

In [ ]:
left

In [ ]:
mercantile.xy_bounds(large_tile_object).left - 250

## georectification exploration

In [ ]:
from affine import Affine
import rasterio
from rasterio.profiles import DefaultGTiffProfile
zoom_level = tile_object.z
bbox_xy = mercantile.xy_bounds(tile_object)
meters_per_pixel = 20037508*2 / 2**(8 + zoom_level)
print("{} level has {} meters per pixel".format(zoom_level, meters_per_pixel))

bbox_xy = mercantile.xy_bounds(tile_object)

transform = Affine(meters_per_pixel, 0, bbox_xy.left-25, 0, -meters_per_pixel, bbox_xy.top)
print(transform)

In [ ]:
super_res_tile.shape

## Write Tile to Image

In [ ]:
testLocation = "/Users/dlindenbaum/cosmiQGit/ml-export-tool/tests/fixtures/my-bucket/spacenet_test/las_vegas_tile_3.tiff"
tile_object_list = [tile_object]
data_list = [super_res_tile]
with rasterio.open(testLocation, 'w', **DefaultGTiffProfile(count=3, height=5000, width=5000, crs="EPSG:3857", transform=transform)) as dst_dataset:
    
    for tile_object, data in zip(tile_object_list, data_list):
        left, bottom, right, top = mercantile.xy_bounds(*tile_object)
        
        dst_window  = rasterio.windows.from_bounds(left, bottom, right, top, transform=transform)
        dst_transform = dst_dataset.window_transform(dst_window)
        print(dst_window)
        print(dst_transform)
        dst_dataset.write(data.astype(np.uint8), window=dst_window) 

In [ ]:
tile_object

In [ ]:
zoom_level = tile_object.z
meters_per_pixel = 20037508*2 / 2**(8 + zoom_level)


In [ ]:
(bbox_xy.right - bbox_xy.left)/meters_per_pixel

In [ ]:
geom = geometry.box(-115.24, 36.1, -115.2, 36.2)
geom = geometry.box(-115.24, 36.1, -115.2, 36.2)

zoom_level = 12
tile_object_list = tile_generator.get_tile_list(geom, zoom=zoom_level)
print("{} z{} Tiles identified".format(len(tile_object_list), zoom_level))

In [ ]:
from ml_export import tile_aggregator

In [ ]:
tile_aggregator.build_cog_from_tiles()

In [ ]:
list(np.asarray([1,2,3])-1)